## RI Model LN Auto V2

In [1]:
!pip install xgboost


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [3]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
import json
import logging
import xgboost as xgb
import ast
import itertools
import scipy.stats as ss
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import chi2
import os
import sys

pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
pd.set_option('display.max_colwidth', None)

In [4]:
sys.path.append('/root/Unified_LN_V2/')
from Utilities import *

In [5]:
Train = pd.read_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/1_LN_Unsec_V2_Train_RI_Data_090424.csv')
Test = pd.read_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/1_LN_Unsec_V2_Test_RI_Data_090424.csv')

Train.shape, Test.shape

((162612, 345), (69691, 345))

In [6]:
Train['approve_flag'].mean(), Test['approve_flag'].mean()

(0.5329188497773842, 0.5327230201891205)

In [7]:
req_cols= ['assetpropevercount', 'sourcenonderogcount06month', 'addronfilecount', 'addrinputcountyratio', 'assetpropnewestsaleprice', 'addrcurrenttaxvalue', 'addrinputavmvalue', 'addrinputavmvalue60month', 'addrinputlastsaleprice', 'addrinputtaxmarketvalue', 'sourcecredheadertimeoldest', 'addrinputphonecount', 'assetpropcurrenttaxtotal', 'addrcurrentavmvalue', 'assetproppurchasetimeoldest', 'addrpreviouslengthofres', 'addrinputlengthofres', 'addrcurrenttaxmarketvalue', 'addrprevioustimenewest', 'addrcurrenttractratio', 'sourcenonderogcount12month', 'assetpropcurrentcount', 'addrinputsubjectcount', 'sourcenonderogcount', 'addrinputtaxvalue', 'assetpropeversoldcount', 'businessassociationtimeoldest', 'addrinputtractratio', 'derogtimenewest', 'addrchangecount24month', 'addrinputtimelastsale', 'educationinstitutionprivate', 'addrinputblockratio', 'businesstitleleadership', 'addrcurrentlengthofres', 'addrcurrentavmratio12monthprior', 'addrcurrentblockratio', 'derogcount', 'profliccount', 'addrinputavmratio60monthprior', 'addrcurrentcountyratio', 'assetpropsaletimeoldest', 'subjectrecordtimeoldest', 'addrcurrenttimenewest', 'proflictypecategory', 'criminalnonfelonytimenewest', 'lienjudgmentdollartotal', 'addrcurrentlastsalesprice', 'addrinputtimenewest', 'evictiontimenewest', 'bankruptcycount', 'addrcurrenttimelastsale', 'addrinputtimeoldest', 'addrlastmovetaxratiodiff', 'criminalnonfelonycount', 'sourcenonderogcount03month', 'addrchangecount03month', 'addrchangecount06month', 'addrchangecount12month', 'addrchangecount60month', 'addrcurrentavmratio60monthprior', 'addrcurrentavmvalue12month', 'addrcurrentavmvalue60month', 'addrcurrentcorrectional', 'addrcurrentdeedmailing', 'addrcurrentphoneservice', 'addrcurrentsubjectowned', 'addrcurrenttimeoldest', 'addrinputavmratio12monthprior', 'addrinputavmvalue12month', 'addrinputdeedmailing', 'addrinputphoneservice', 'addrinputproblems', 'addrinputsubjectowned', 'addrlastmoveecontrajectory', 'addronfilecollege', 'addronfilecorrectional', 'addronfilehighrisk', 'addrpreviouscorrectional', 'addrprevioussubjectowned', 'addrprevioustimeoldest', 'alertregulatorycondition', 'assetownership', 'assetpersonal', 'assetpersonalcount', 'assetprop', 'assetpropnewestmortgagetype', 'assetproppurchasecount12month', 'assetproppurchasetimenewest', 'assetpropsalepurchaseratio', 'assetpropsaletimenewest', 'assetpropsoldcount12month', 'bankruptcychapter', 'bankruptcycount24month', 'bankruptcydismissed24month', 'bankruptcystatus', 'bankruptcytimenewest', 'businessassociation', 'criminalfelonycount', 'criminalfelonycount12month', 'criminalfelonytimenewest', 'criminalnonfelonycount12month', 'derogcount12month', 'educationattendance', 'educationevidence', 'educationinstitutionrating', 'educationprogramattended', 'evictioncount', 'evictioncount12month', 'inquiryauto12month', 'inquirybanking12month', 'inquirycollections12month', 'inquirynonshortterm12month', 'inquiryshortterm12month', 'inquirytelcom12month', 'lienjudgmentcount', 'lienjudgmentcount12month', 'lienjudgmentcourtcount', 'lienjudgmentforeclosurecount', 'lienjudgmentothercount', 'lienjudgmentsmallclaimscount', 'lienjudgmenttaxcount', 'lienjudgmenttimenewest', 'shorttermloanrequest', 'shorttermloanrequest12month', 'shorttermloanrequest24month', 'sourcecredheadertimenewest', 'sourcevoterregistration', 'subjectnewestrecord12month', 'subjectrecordtimenewest']
len(req_cols)

130

In [8]:
all_cols= ['unique_id', 'client', 'app_date', 'approve_flag', 'funded_flag', 'fico_pb', 'fico_cb', 'bad_flag', 'product', 'auto_score', 'bankcard_score', 'Race', 'Gender', 'age_cuts'] + req_cols
len(all_cols)

144

In [9]:
Train[all_cols].head()

,unique_id,client,app_date,approve_flag,funded_flag,fico_pb,fico_cb,bad_flag,product,auto_score,bankcard_score,Race,Gender,age_cuts,assetpropevercount,sourcenonderogcount06month,addronfilecount,addrinputcountyratio,assetpropnewestsaleprice,addrcurrenttaxvalue,addrinputavmvalue,addrinputavmvalue60month,addrinputlastsaleprice,addrinputtaxmarketvalue,sourcecredheadertimeoldest,addrinputphonecount,assetpropcurrenttaxtotal,addrcurrentavmvalue,assetproppurchasetimeoldest,addrpreviouslengthofres,addrinputlengthofres,addrcurrenttaxmarketvalue,addrprevioustimenewest,addrcurrenttractratio,sourcenonderogcount12month,assetpropcurrentcount,addrinputsubjectcount,sourcenonderogcount,addrinputtaxvalue,assetpropeversoldcount,businessassociationtimeoldest,addrinputtractratio,derogtimenewest,addrchangecount24month,addrinputtimelastsale,educationinstitutionprivate,addrinputblockratio,businesstitleleadership,addrcurrentlengthofres,addrcurrentavmratio12monthprior,addrcurrentblockratio,derogcount,profliccount,addrinputavmratio60monthprior,addrcurrentcountyratio,assetpropsaletimeoldest,subjectrecordtimeoldest,addrcurrenttimenewest,proflictypecategory,criminalnonfelonytimenewest,lienjudgmentdollartotal,addrcurrentlastsalesprice,addrinputtimenewest,evictiontimenewest,bankruptcycount,addrcurrenttimelastsale,addrinputtimeoldest,addrlastmovetaxratiodiff,criminalnonfelonycount,sourcenonderogcount03month,addrchangecount03month,addrchangecount06month,addrchangecount12month,addrchangecount60month,addrcurrentavmratio60monthprior,addrcurrentavmvalue12month,addrcurrentavmvalue60month,addrcurrentcorrectional,addrcurrentdeedmailing,addrcurrentphoneservice,addrcurrentsubjectowned,addrcurrenttimeoldest,addrinputavmratio12monthprior,addrinputavmvalue12month,addrinputdeedmailing,addrinputphoneservice,addrinputproblems,addrinputsubjectowned,addrlastmoveecontrajectory,addronfilecollege,addronfilecorrectional,addronfilehighrisk,addrpreviouscorrectional,addrprevioussubjectowned,addrprevioustimeoldest,alertregulatorycondition,assetownership,assetpersonal,assetpersonalcount,assetprop,assetpropnewestmortgagetype,assetproppurchasecount12month,assetproppurchasetimenewest,assetpropsalepurchaseratio,assetpropsaletimenewest,assetpropsoldcount12month,bankruptcychapter,bankruptcycount24month,bankruptcydismissed24month,bankruptcystatus,bankruptcytimenewest,businessassociation,criminalfelonycount,criminalfelonycount12month,criminalfelonytimenewest,criminalnonfelonycount12month,derogcount12month,educationattendance,educationevidence,educationinstitutionrating,educationprogramattended,evictioncount,evictioncount12month,inquiryauto12month,inquirybanking12month,inquirycollections12month,inquirynonshortterm12month,inquiryshortterm12month,inquirytelcom12month,lienjudgmentcount,lienjudgmentcount12month,lienjudgmentcourtcount,lienjudgmentforeclosurecount,lienjudgmentothercount,lienjudgmentsmallclaimscount,lienjudgmenttaxcount,lienjudgmenttimenewest,shorttermloanrequest,shorttermloanrequest12month,shorttermloanrequest24month,sourcecredheadertimenewest,sourcevoterregistration,subjectnewestrecord12month,subjectrecordtimenewest
0,107884,CPM,2022-04-01,1.0,1.0,523.0,NaN,0.0,Unsecured,693.0,716.0,missing,F,Age below 62,0.0,2.0,2.0,0.62,-1.0,0.0,56760.0,38549.0,-1.0,0.0,359.0,1.0,-1.0,56760.0,-1.0,65.0,213.0,0.0,15.0,0.65,2.0,0.0,2.0,4.0,0.0,0.0,-1.0,0.65,-1.0,0.0,-1.0,-1.0,0.62,-1.0,213.0,1.31,0.62,0.0,0.0,1.47,0.62,-1.0,359.0,1.0,0.0,-1.0,0.0,-1.0,1.0,-1.0,0.0,-1.0,214.0,-1.0,0.0,2.0,0.0,0.0,0.0,0.0,1.47,43319.0,38549.0,0.0,0.0,1.0,0.0,214.0,1.31,43319.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,-1.0,80.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1,SYM662540,Numerica,2018-05-31,1.0,1.0,615.0,NaN,0.0,Unsecured Closed End,647.0,669.0,white,F,Age below 62,0.0,1.0,3.0,0.00,-1.0,0.0,-1.0,-1.0,-1.0,0.0,30.0,0.0,-1.0,-1.0,-1.0,12.0,-1.0,0.0,1.0,0.00,1.

In [10]:
Train['Race'].value_counts(dropna= False), Train['Gender'].value_counts(dropna= False), Train['age_cuts'].value_counts(dropna= False)

(Race
 white       64784
 missing     63396
 black       17707
 hispanic    14547
 api          2013
 multiple       87
 aian           72
 others          6
 Name: count, dtype: int64,
 Gender
 M    76401
 F    71235
 I    14976
 Name: count, dtype: int64,
 age_cuts
 Age below 62       138145
 Age 62 and over     17603
 missing              6864
 Name: count, dtype: int64)

In [11]:
Test['Race'].value_counts(dropna= False), Test['Gender'].value_counts(dropna= False), Test['age_cuts'].value_counts(dropna= False)

(Race
 white       27844
 missing     27037
 black        7614
 hispanic     6265
 api           868
 multiple       35
 aian           28
 Name: count, dtype: int64,
 Gender
 M    32896
 F    30339
 I     6456
 Name: count, dtype: int64,
 age_cuts
 Age below 62       58934
 Age 62 and over     7740
 missing             3017
 Name: count, dtype: int64)

### Base Model RI

In [12]:
!pip install hyperopt


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [13]:
import numpy as np
import pandas as pd
from hyperopt import Trials, STATUS_OK, tpe, hp, fmin
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
import h2o
from h2o.estimators import H2OXGBoostEstimator
from hyperopt.pyll import scope as ho_scope
from hyperopt.pyll.stochastic import sample as ho_sample
import warnings
warnings.filterwarnings('ignore')

In [14]:
### Starting H2o Session
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.2" 2019-01-15; OpenJDK Runtime Environment 18.9 (build 11.0.2+9); OpenJDK 64-Bit Server VM 18.9 (build 11.0.2+9, mixed mode)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp0w1gvl5d
  JVM stdout: /tmp/tmp0w1gvl5d/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp0w1gvl5d/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.1
H2O_cluster_version_age:,26 days
H2O_cluster_name:,H2O_from_python_unknownUser_94kdg8
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.486 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [15]:
Train= Train[all_cols]
Test= Test[all_cols]

Train.shape, Test.shape

((162612, 144), (69691, 144))

In [16]:
### Convert the Variable into categorical
var_to_cat= '''businesstitleleadership
proflictypecategory
addrcurrentdeedmailing
addrcurrentphoneservice
addrcurrentsubjectowned
addrinputdeedmailing
addrinputphoneservice
addrinputproblems
addrinputsubjectowned
addrlastmoveecontrajectory
addronfilecollege
addronfilecorrectional
addronfilehighrisk
addrprevioussubjectowned
alertregulatorycondition
assetownership
assetpersonal
assetprop
assetpropnewestmortgagetype
bankruptcychapter
bankruptcystatus
businessassociation
educationattendance
educationevidence
educationinstitutionrating
educationprogramattended
inquiryauto12month
inquirybanking12month
inquirycollections12month
inquirynonshortterm12month
inquiryshortterm12month
inquirytelcom12month
shorttermloanrequest
shorttermloanrequest12month
shorttermloanrequest24month
subjectnewestrecord12month'''.splitlines()
len(var_to_cat)

36

In [17]:
var_to_cat= list(set(var_to_cat))
len(var_to_cat)

36

In [18]:
Train[req_cols].isnull().sum()

assetpropevercount                 0
sourcenonderogcount06month         0
addronfilecount                    0
addrinputcountyratio               0
assetpropnewestsaleprice           0
addrcurrenttaxvalue                0
addrinputavmvalue                  0
addrinputavmvalue60month           0
addrinputlastsaleprice             0
addrinputtaxmarketvalue            0
sourcecredheadertimeoldest         0
addrinputphonecount                0
assetpropcurrenttaxtotal           0
addrcurrentavmvalue                0
assetproppurchasetimeoldest        0
addrpreviouslengthofres            0
addrinputlengthofres               0
addrcurrenttaxmarketvalue          0
addrprevioustimenewest             0
addrcurrenttractratio              0
sourcenonderogcount12month         0
assetpropcurrentcount              0
addrinputsubjectcount              0
sourcenonderogcount                0
addrinputtaxvalue                  0
assetpropeversoldcount             0
businessassociationtimeoldest      0
a

In [19]:
# for var in var_to_cat:
#     print(var)
#     print(Train[var].value_counts(dropna= False))

In [20]:
for var in var_to_cat:
    Train[var]= Train[var].astype(int).astype(str)
    Test[var]= Test[var].astype(int).astype(str)

In [21]:
Train['wt2']= 1
Test['wt2']= 1

train_h2o = h2o.H2OFrame(Train)
test_h2o = h2o.H2OFrame(Test)
print(Train.shape,train_h2o.shape,Test.shape,test_h2o.shape)
bad_col = 'approve_flag'
Train[bad_col].mean(),Test[bad_col].mean()

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
(162612, 145) (162612, 145) (69691, 145) (69691, 145)


(0.5329188497773842, 0.5327230201891205)

### Baseline

In [22]:
# config= pd.read_csv('/home/pradeep/000_LDA_March2024/configfile.csv')
unique_cols = ['unique_id', 'client']
wt_col = 'wt2'
folds = 10
# predictors = train_h2o.columns
# req_cols = config.dropna().Feature_Name.to_list()
model_cols = unique_cols + [wt_col] + [bad_col] + req_cols

# sample_h2o=train_h2o[model_cols+['age_cuts','Race','Gender']]#,'mmct_tag']]
# sample_h2o[bad_col] = sample_h2o[bad_col].asfactor()
# print(sample_h2o.shape)
# print(len(req_cols))

In [23]:
bad_col

'approve_flag'

### Run the Code

In [25]:
product= 'Unsec'

In [29]:
train_h2o[bad_col] = train_h2o[bad_col].asfactor()
test_h2o[bad_col] = test_h2o[bad_col].asfactor()

In [30]:
xgbm_ri_model = H2OXGBoostEstimator(
    ntrees=2000,    
    learn_rate=0.01,
    max_depth=2,    
    model_id=f"RI_{product}_LN_V2_1",
    distribution = "bernoulli",    
    seed=42,
    nfolds = folds)#,    ignored_columns = unique_cols)

In [31]:
xgbm_ri_model.train(y=bad_col, x= req_cols, training_frame=train_h2o[model_cols], weights_column =wt_col)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: RI_Unsec_LN_V2_1


Model Summary: 
    number_of_trees
--  -----------------
    2000

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.2103346060758763
RMSE: 0.4586225093427887
LogLoss: 0.6034568369340186
Mean Per-Class Error: 0.3770034360806913
AUC: 0.7196306558998417
AUCPR: 0.7156279872849437
Gini: 0.4392613117996833

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3990572741970165
       0      1       Error    Rate
-----  -----  ------  -------  ------------------
0      24444  51509   0.6782   (51509.0/75953.0)
1      6572   80087   0.0758   (6572.0/86659.0)
Total  31016  131596  0.3572   (58081.0/162612.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.399057     0.733885  276
max f2                       0.231329     0.863195  354
max f0point5                 0.54041      0.680497  188
max accuracy                 0.502907     0.666089  213
max precision                0.859961     0.947368  0
max recall                   0.0785512    1         388
max specificity              0.859961     0.999987  0
max absolute_mcc             0.483209     0.327258  226
max min_per_class_accuracy   0.562654     0.655754  173
max mean_per_class_accuracy  0.525073     0.658892  198
max tns                      0.859961     75952     0
max fns                      0.859961     86641     0
max fps                      0.00127567   75953     399
max tps                      0.0785512    86659     388
max tnr                      0.859961     0.999987  0
max fnr                      0.859961     0.999792  0
max fpr                      0.00127567   1         399
max tpr                      0.0785512    1         388

Gains/Lift Table: Avg response rate: 53.29 %, avg score: 53.29 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100116                   0.815262           1.61136   1.61136            0.858722         0.829174  0.858722                    0.829174            0.0161322       0.0161322                  61.1357   61.1357            0.013104
2        0.0200047                   0.79974            1.54274   1.57708            0.822154         0.806907  0.840455                    0.818051            0.0154167       0.0315489                  54.2737   57.7079            0.0247158
3        0.0300039                   0.787391           1.49794   1.5507             0.798278         0.793348  0.826399                    0.809818            0.0149782       0.0465272                  49.7935   55.0703            0.0353756
4        0.0400032                   0.777046           1.46908   1.5303             0.782903         0.782012  0.815527                    0.802868            0.0146898       0.061217                   46.9085   53.0302            0.0454177
5        0.0500025                   0.767959           1.47716   1.51968            0.787208         0.772479  0.809863                    0.796791            0.0147705       0.0759875                  47.7163   51.9675            0.0556328
6        0.100005                    0.729309           1.45506   1.48737            0.775427         0.74762   0.792645                    0.772206            0.0727564       0.148744                   45.5057   48.7366            0.104348
7        0.150001                    0.701837           1.38923   1.45466            0.74

##### Train Predictions

In [33]:
col_tuple_pass= ["unique_id", 'client', "wt2", 'fico_pb', 'approve_flag', 'funded_flag', 'bad_flag', 'Race', 'Gender', 'age_cuts']

In [34]:
def get_risk_rank(model, data, data_h2o, target, cols_tuple, score_col= 'p1', weight= 'RECWGT', asc= False):
    
    if data_h2o!= None:
        predict = model.predict(data_h2o)
    else:
        predict = model.predict(h2o.H2OFrame(data))
    p =predict.as_data_frame()
    dd = data.loc[:, col_tuple_pass]
    dd = pd.DataFrame(dd)
    dd = dd.reset_index()
    dd['RECWGT'] = dd["wt2"]
    dd['conwt'] = 1
    dd[target] = dd[bad_col] 
    pred = pd.concat([dd.loc[:, cols_tuple + ['RECWGT', 'conwt']],p],axis=1)
    print(pred[weight].sum(),((pred[weight]*pred[target]).sum()),pred[weight].max())

    data_ks_cap5= rnkOrderingN(pred, target, weight, score_col, 5, asc)
    display(data_ks_cap5[0])
    print(data_ks_cap5[1])
    # data_ks_cap10= rnkOrderingN(pred, target, weight, score_col, 10, asc)
    # display(data_ks_cap10[0])
    # print(data_ks_cap10[1])

In [35]:
get_risk_rank(model= xgbm_ri_model, data= Train, data_h2o= train_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
162612 86659.0 1


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS
0,0.680238,0.862986,0.737535,32523,7845.0,24678.0,75.88,20.0,10.33,28.48,18.15
1,0.606160,0.680229,0.641995,32522,10879.0,21643.0,66.55,40.0,24.65,53.45,28.80
2,0.517135,0.606159,0.564352,32523,14012.0,18511.0,56.92,60.0,43.10,74.81,31.71
3,0.405086,0.517126,0.465619,32522,17852.0,14670.0,45.11,80.0,66.60,91.74,25.14
4,0.000494,0.405086,0.255090,32522,25365.0,7157.0,22.01,100.0,100.00,100.00,0.00


{'Gini': 41.52, 'KS': 31.71}


In [36]:
get_risk_rank(model= xgbm_ri_model, data= Test, data_h2o= test_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
69691 37126.0 1


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS
0,0.680787,0.862849,0.738294,13939,3435.0,10504.0,75.36,20.0,10.55,28.29,17.74
1,0.606300,0.680784,0.642493,13938,4582.0,9356.0,67.13,40.0,24.62,53.49,28.87
2,0.517266,0.606300,0.564664,13938,5993.0,7945.0,57.00,60.0,43.02,74.89,31.87
3,0.406043,0.517261,0.466144,13938,7726.0,6212.0,44.57,80.0,66.75,91.63,24.88
4,0.000529,0.406042,0.255618,13938,10829.0,3109.0,22.31,100.0,100.00,100.00,0.00


{'Gini': 41.349999999999994, 'KS': 31.87}


#### Reading OOT Data

In [37]:
Oot= pd.read_csv('/root/Unified_LN_V2/LN_Unsec_V2/Data/1_LN_Unsec_V2_OOT_Data_090424.csv')
Oot.shape

(58076, 342)

In [38]:
Oot['Race']= Oot['Borrower_Race_Prim_BISG']
Oot['Gender']= Oot['borrower_gender']
Oot['age_cuts']= np.where(Oot['applicant_age'].isnull(), 'missing',
                         np.where(Oot['applicant_age']< 62, 'Age below 62',
                                  np.where(Oot['applicant_age']>= 62, 'Age 62 and over', 'missing')))

Oot['age_cuts'].value_counts(dropna= False), Oot['applicant_age'].describe()

(age_cuts
 Age below 62       49425
 Age 62 and over     6190
 missing             2461
 Name: count, dtype: int64,
 count    55615.000000
 mean        42.427140
 std         14.910266
 min          0.002736
 25%         29.762312
 50%         42.000000
 75%         51.922139
 max         99.948016
 Name: applicant_age, dtype: float64)

In [39]:
Train.shape, Test.shape, Oot.shape

((162612, 145), (69691, 145), (58076, 345))

In [40]:
Oot['Race'].value_counts(dropna= False), Oot['age_cuts'].value_counts(dropna= False), Oot['Gender'].value_counts(dropna= False)

(Race
 white       22956
 missing     22522
 black        6579
 hispanic     5229
 api           733
 multiple       29
 aian           28
 Name: count, dtype: int64,
 age_cuts
 Age below 62       49425
 Age 62 and over     6190
 missing             2461
 Name: count, dtype: int64,
 Gender
 M    27081
 F    25612
 I     5383
 Name: count, dtype: int64)

In [41]:
Oot[all_cols].head(2)

,unique_id,client,app_date,approve_flag,funded_flag,fico_pb,fico_cb,bad_flag,product,auto_score,bankcard_score,Race,Gender,age_cuts,assetpropevercount,sourcenonderogcount06month,addronfilecount,addrinputcountyratio,assetpropnewestsaleprice,addrcurrenttaxvalue,addrinputavmvalue,addrinputavmvalue60month,addrinputlastsaleprice,addrinputtaxmarketvalue,sourcecredheadertimeoldest,addrinputphonecount,assetpropcurrenttaxtotal,addrcurrentavmvalue,assetproppurchasetimeoldest,addrpreviouslengthofres,addrinputlengthofres,addrcurrenttaxmarketvalue,addrprevioustimenewest,addrcurrenttractratio,sourcenonderogcount12month,assetpropcurrentcount,addrinputsubjectcount,sourcenonderogcount,addrinputtaxvalue,assetpropeversoldcount,businessassociationtimeoldest,addrinputtractratio,derogtimenewest,addrchangecount24month,addrinputtimelastsale,educationinstitutionprivate,addrinputblockratio,businesstitleleadership,addrcurrentlengthofres,addrcurrentavmratio12monthprior,addrcurrentblockratio,derogcount,profliccount,addrinputavmratio60monthprior,addrcurrentcountyratio,assetpropsaletimeoldest,subjectrecordtimeoldest,addrcurrenttimenewest,proflictypecategory,criminalnonfelonytimenewest,lienjudgmentdollartotal,addrcurrentlastsalesprice,addrinputtimenewest,evictiontimenewest,bankruptcycount,addrcurrenttimelastsale,addrinputtimeoldest,addrlastmovetaxratiodiff,criminalnonfelonycount,sourcenonderogcount03month,addrchangecount03month,addrchangecount06month,addrchangecount12month,addrchangecount60month,addrcurrentavmratio60monthprior,addrcurrentavmvalue12month,addrcurrentavmvalue60month,addrcurrentcorrectional,addrcurrentdeedmailing,addrcurrentphoneservice,addrcurrentsubjectowned,addrcurrenttimeoldest,addrinputavmratio12monthprior,addrinputavmvalue12month,addrinputdeedmailing,addrinputphoneservice,addrinputproblems,addrinputsubjectowned,addrlastmoveecontrajectory,addronfilecollege,addronfilecorrectional,addronfilehighrisk,addrpreviouscorrectional,addrprevioussubjectowned,addrprevioustimeoldest,alertregulatorycondition,assetownership,assetpersonal,assetpersonalcount,assetprop,assetpropnewestmortgagetype,assetproppurchasecount12month,assetproppurchasetimenewest,assetpropsalepurchaseratio,assetpropsaletimenewest,assetpropsoldcount12month,bankruptcychapter,bankruptcycount24month,bankruptcydismissed24month,bankruptcystatus,bankruptcytimenewest,businessassociation,criminalfelonycount,criminalfelonycount12month,criminalfelonytimenewest,criminalnonfelonycount12month,derogcount12month,educationattendance,educationevidence,educationinstitutionrating,educationprogramattended,evictioncount,evictioncount12month,inquiryauto12month,inquirybanking12month,inquirycollections12month,inquirynonshortterm12month,inquiryshortterm12month,inquirytelcom12month,lienjudgmentcount,lienjudgmentcount12month,lienjudgmentcourtcount,lienjudgmentforeclosurecount,lienjudgmentothercount,lienjudgmentsmallclaimscount,lienjudgmenttaxcount,lienjudgmenttimenewest,shorttermloanrequest,shorttermloanrequest12month,shorttermloanrequest24month,sourcecredheadertimenewest,sourcevoterregistration,subjectnewestrecord12month,subjectrecordtimenewest
0,515342,Bethpage,2018-09-22,0.0,NaN,639.0,NaN,NaN,Cards,721.0,787.0,white,F,Age below 62,0.0,2.0,2.0,0.83,-1.0,0.0,287705.0,335000.0,-1.0,0.0,71.0,0.0,-1.0,287705.0,-1.0,70.0,49.0,0.0,1.0,0.96,2.0,0.0,5.0,2.0,0.0,0.0,-1.0,0.96,-1.0,0.0,-1.0,-1.0,0.94,-1.0,49.0,0.83,0.94,0.0,0.0,0.86,0.83,-1.0,71.0,1.0,-1.0,-1.0,0.0,-1.0,1.0,-1.0,0.0,-1.0,50.0,-1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.86,348243.0,335000.0,0.0,1.0,0.0,0.0,50.0,0.83,348243.0,1.0,0.0,0.0,0.0,9.0,1.0,0.0,0.0,0.0,1.0,71.0,1.0,0.0,0.0,0.0,0.0,-1.0,0.0,-1.0,0.00,-1.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0
1,39139,HFCU,2020-05-06,1.0,1.0,624.0,NaN,0.0,Personal,732.0,806.0,missing,I,missing,1.0,1.0,9.0,1.58,129750.0,0.0,388031.0,369020.0,-1.0,0.0,294.0,1.0,-1.0,388031.0,118.0,108.0,125.0,0.0,114.0,1.90,1.0,

In [42]:
Oot= Oot[all_cols]

for var in var_to_cat:
    Oot[var]= Oot[var].astype(int).astype(str)

In [43]:
Oot['wt2']= 1
oot_h2o = h2o.H2OFrame(Oot)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [44]:
get_risk_rank(model= xgbm_ri_model, data= Oot, data_h2o= oot_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
58076 31097.0 1


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS
0,0.679758,0.862100,0.737030,11616,2769.0,8847.0,76.16,20.0,10.26,28.45,18.19
1,0.604872,0.679755,0.641016,11615,3904.0,7711.0,66.39,40.0,24.73,53.25,28.52
2,0.516240,0.604863,0.562962,11615,4915.0,6700.0,57.68,60.0,42.95,74.79,31.84
3,0.405907,0.516239,0.465601,11615,6374.0,5241.0,45.12,80.0,66.58,91.65,25.07
4,0.000494,0.405867,0.256482,11615,9017.0,2598.0,22.37,100.0,100.00,100.00,0.00


{'Gini': 41.449999999999996, 'KS': 31.84}


In [46]:
var_imp = xgbm_ri_model._model_json['output']['variable_importances'].as_data_frame()
var_imp['Var_unique'] = var_imp['variable'].str.split('.', expand=True)[0]
var_imp_list_baseline1 = list(var_imp['Var_unique'].unique())

print(var_imp_list_baseline1, len(var_imp_list_baseline1))

['addrinputcountyratio', 'inquirynonshortterm12month', 'sourcevoterregistration', 'inquiryshortterm12month', 'subjectrecordtimeoldest', 'assetownership', 'assetpropevercount', 'derogtimenewest', 'addrinputtaxvalue', 'addrinputsubjectcount', 'addrinputavmvalue60month', 'assetpropcurrenttaxtotal', 'inquiryauto12month', 'addrinputavmvalue', 'lienjudgmentothercount', 'addrcurrentcountyratio', 'addronfilecount', 'addrcurrentavmvalue', 'inquirybanking12month', 'inquirycollections12month', 'addrcurrenttaxvalue', 'addrcurrentavmvalue60month', 'alertregulatorycondition', 'derogcount12month', 'addrinputtimeoldest', 'addrinputlengthofres', 'sourcecredheadertimeoldest', 'addrinputphonecount', 'addrinputdeedmailing', 'bankruptcytimenewest', 'sourcenonderogcount', 'addrlastmoveecontrajectory', 'addrcurrentdeedmailing', 'bankruptcycount24month', 'lienjudgmentcourtcount', 'inquirytelcom12month', 'lienjudgmenttaxcount', 'addrprevioustimeoldest', 'assetproppurchasetimenewest', 'addrinputavmratio60monthp

In [47]:
var_imp

,variable,relative_importance,scaled_importance,percentage,Var_unique
0,addrinputcountyratio,489037.156250,1.000000,0.341979,addrinputcountyratio
1,inquirynonshortterm12month,196287.781250,0.401376,0.137262,inquirynonshortterm12month
2,sourcevoterregistration,144406.375000,0.295287,0.100982,sourcevoterregistration
3,inquiryshortterm12month,87823.546875,0.179585,0.061414,inquiryshortterm12month
4,subjectrecordtimeoldest,71039.117188,0.145263,0.049677,subjectrecordtimeoldest
5,assetownership,41842.476562,0.085561,0.029260,assetownership
6,assetpropevercount,37581.820312,0.076849,0.026281,assetpropevercount
7,derogtimenewest,36966.335938,0.075590,0.025850,derogtimenewest
8,addrinputtaxvalue,33653.289062,0.068815,0.023533,addrinputtaxvalue
9,addrinputsubjectcount,29392.808594,0.060103,0.020554,addrinputsubjectcount


## HyperOpt

In [48]:
# sample_1= sample.sample(frac= 0.1, random_state= 101)
# sample_1.shape

In [49]:
# sample_1_h2o= h2o.H2OFrame(sample_1)
# sample_1_h2o.shape, sample_1.shape

In [50]:
# sample_1_h2o[bad_col]= sample_1_h2o[bad_col].asfactor()

In [51]:
train_h2o[bad_col] = train_h2o[bad_col].asfactor()
test_h2o[bad_col] = test_h2o[bad_col].asfactor()
oot_h2o[bad_col] = oot_h2o[bad_col].asfactor()
# col_list = var_imp_list_baseline1
col_list= ['addrinputcountyratio', 'inquirynonshortterm12month', 'sourcevoterregistration', 'inquiryshortterm12month', 'subjectrecordtimeoldest', 'assetownership', 'assetpropevercount', 'derogtimenewest', 'addrinputtaxvalue', 'addrinputsubjectcount', 'addrinputavmvalue60month', 'assetpropcurrenttaxtotal', 'inquiryauto12month', 'addrinputavmvalue', 'lienjudgmentothercount', 'addrcurrentcountyratio', 'addronfilecount', 'addrcurrentavmvalue', 'inquirybanking12month', 'inquirycollections12month', 'addrcurrenttaxvalue', 'addrcurrentavmvalue60month', 'alertregulatorycondition', 'derogcount12month', 'addrinputtimeoldest', 'addrinputlengthofres', 'sourcecredheadertimeoldest', 'addrinputphonecount', 'addrinputdeedmailing', 'bankruptcytimenewest', 'sourcenonderogcount', 'addrlastmoveecontrajectory', 'addrcurrentdeedmailing', 'bankruptcycount24month', 'lienjudgmentcourtcount', 'inquirytelcom12month', 'lienjudgmenttaxcount', 'addrprevioustimeoldest', 'assetproppurchasetimenewest', 'addrinputavmratio60monthprior', 'addrinputavmvalue12month', 'addrcurrentavmratio60monthprior', 'addrlastmovetaxratiodiff', 'addrcurrentavmvalue12month', 'lienjudgmentdollartotal', 'addrprevioustimenewest', 'assetpropsaletimenewest', 'assetpersonal', 'addrcurrentblockratio', 'assetpropsalepurchaseratio', 'addrcurrentphoneservice', 'addrinputavmratio12monthprior', 'educationinstitutionrating', 'lienjudgmenttimenewest', 'shorttermloanrequest', 'addrcurrentlengthofres', 'addrpreviouslengthofres', 'sourcenonderogcount12month', 'addrchangecount60month', 'addrinputlastsaleprice', 'addrcurrenttimelastsale', 'evictiontimenewest', 'assetproppurchasecount12month', 'addrinputtimelastsale', 'addrcurrentavmratio12monthprior', 'businessassociationtimeoldest', 'addrcurrentlastsalesprice', 'addrinputsubjectowned', 'addrcurrenttractratio', 'assetpropnewestsaleprice', 'bankruptcydismissed24month', 'addrcurrentsubjectowned', 'addrinputblockratio']
len(col_list)
# 73
# col_list = ['inquiryauto12month', 'derogtimenewest', 'addrinputtaxmarketvalue', 'assetownership', 'addrinputsubjectowned', 'assetpropevercount', 'assetpropcurrenttaxtotal', 'inquiryshortterm12month', 'sourcenonderogcount', 'sourcenonderogcount12month', 'addrinputavmvalue', 'inquirynonshortterm12month', 'addrinputcountyratio', 'addrprevioustimeoldest', 'addrinputavmvalue12month', 'addrinputlengthofres', 'assetpropnewestmortgagetype', 'subjectrecordtimeoldest', 'addronfilecount', 'addrcurrentavmvalue60month', 'addrcurrentavmvalue12month', 'inquirycollections12month', 'sourcecredheadertimeoldest', 'assetpropnewestsaleprice', 'addrinputsubjectcount', 'bankruptcycount24month', 'addrinputtimenewest', 'educationevidence', 'addrinputproblems', 'addrcurrentphoneservice', 'bankruptcychapter', 'derogcount12month', 'addrcurrentcountyratio', 'addrcurrenttaxmarketvalue', 'alertregulatorycondition', 'educationprogramattended', 'assetpersonal', 'inquirytelcom12month', 'addrcurrentlengthofres', 'addrcurrentavmratio60monthprior', 'bankruptcytimenewest', 'inquirybanking12month', 'addrinputavmvalue60month', 'addrinputtaxvalue', 'derogcount', 'bankruptcycount', 'evictiontimenewest', 'lienjudgmenttaxcount', 'lienjudgmentcourtcount', 'addrprevioustimenewest', 'assetpropsalepurchaseratio', 'proflictypecategory', 'assetpropeversoldcount', 'addrcurrenttimenewest', 'addrinputtimelastsale', 'addrcurrenttaxvalue', 'lienjudgmentothercount', 'assetpersonalcount', 'addrlastmovetaxratiodiff', 'assetpropsaletimenewest', 'addrinputlastsaleprice', 'addrcurrentavmratio12monthprior', 'addrpreviouslengthofres', 'educationinstitutionrating', 'addrcurrentavmvalue', 'bankruptcydismissed24month', 'addrinputavmratio60monthprior', 'assetproppurchasetimenewest', 'addrinputdeedmailing', 'addrcurrenttractratio', 'addrinputtractratio', 'addrcurrenttimelastsale', 'sourcevoterregistration', 'addrinputblockratio', 'addrcurrentlastsalesprice', 'assetpropsaletimeoldest', 'sourcenonderogcount06month', 'addrcurrenttimeoldest', 'addrinputavmratio12monthprior', 'lienjudgmentsmallclaimscount', 'addrinputtimeoldest', 'bankruptcystatus', 'businessassociationtimeoldest', 'assetproppurchasecount12month', 'addrprevioussubjectowned', 'lienjudgmenttimenewest']

73

In [52]:
from IPython.display import clear_output


def get_predictions(data_h2o, bad_col, wt_col, classifier):
    predict = classifier.predict(data_h2o)
    p = predict.as_data_frame()
    dd = pd.DataFrame(data_h2o[:, unique_cols + [bad_col] + [wt_col] + ['age_cuts','Race','Gender']].as_data_frame())
    dd['RECWGT'] = dd[wt_col]
    dd['con_wt'] = 1
    dd['bad'] = dd[bad_col]
    pred = pd.concat([dd.loc[:, ('bad', "RECWGT", 'con_wt', 'age_cuts','Race','Gender')], p], axis=1)
    return pred


def find_cutoff(pred):
    sorted_pred = pred.sort_values(by='p1').reset_index(drop= True)
    cum_bad = sorted_pred['bad'].cumsum()
    bad_rate= sorted_pred['bad'].mean()
    cum_p1 = sorted_pred['p1'].cumsum()
    total_bad = cum_bad.iloc[-1]
#     total_bad= bad_rate
    cutoff_idx = (cum_p1 >= total_bad).idxmax()
    cutoff = sorted_pred.loc[cutoff_idx, 'p1']
    
    print('Pred Description')
    display(pd.DataFrame(pred[['p1']].describe()).T)
    print('Last Row')
    display(pd.DataFrame(sorted_pred.iloc[-1]).T)
    print('Cutoff Index Row')
    display(pd.DataFrame(sorted_pred.iloc[cutoff_idx]).T)
    
    print({'total_bad': total_bad, 'bad_rate': bad_rate, 'cutoff': cutoff, 'total_predicted': cum_p1.iloc[-1], 'max_prob': pred['p1'].max()})
    
    return cutoff

def Equal_Approval_Cutoff_Barsath(pred):
    pred= pred.copy()
    actual_approved_count= pred['bad'].sum()
    pred= pred.sort_values(by= ['p1'])
    pred['cumsum']= pred['con_wt'].cumsum()
    pred['approved']= np.where(pred['cumsum']<= actual_approved_count, 1, 0)
    return pred #cutoff, original_dataset

def find_cutoff_2(pred):
    pred= pred.copy()
    print('Pred Description')
    display(pd.DataFrame(pred[['p1']].describe()).T)
    prev_min = 1
    for k in range(1,100,1):
        pred["model_approval"] = [1 if i>k/100 else 0 for i in pred['p1']]
        a,b = pred.describe().loc['mean',['model_approval','bad']]
        if prev_min < abs(a-b):
            pred["model_approval"] = [1 if i>(k-1)/100 else 0 for i in pred['p1']]
            break
        else:
            prev_min = abs(a-b)
    print({'cutoff': (k-1)/100})
    return (k-1)/100

def air_calc(pred,prefix):
    air = {}
    target_control_dict = {
        'age_cuts': {
            'target': ['Age 62 and over'],
            'control': ['Age below 62']
        },
        'Race': {
            'target': ['black', 'hispanic'],
            'control': ['white']
        },
        'Race': {
            'target': ['black'],
            'control': ['white']
        },
        'Race': {
            'target': ['hispanic'],
            'control': ['white']
        },
        'Gender': {
            'target': ['F'],
            'control': ['M']
        },
        # 'mmct_tag': {
        #     'target': ['mmct'],
        #     'control': ['non-mmct']
        # }
    }
    
    for var in target_control_dict:
        target_list = target_control_dict[var]['target']
        control_list = target_control_dict[var]['control']
        p_target_list= ','.join(target_list)
        ttd_target= pred[pred[var].isin(target_list)]['con_wt'].sum()
        approved_target= pred[pred[var].isin(target_list)]['approved'].multiply(pred[pred[var].isin(target_list)]['con_wt']).sum()
        ttd_control= pred[pred[var].isin(control_list)]['con_wt'].sum()
        approved_control= pred[pred[var].isin(control_list)]['approved'].multiply(pred[pred[var].isin(control_list)]['con_wt']).sum()
        app_pct_target= round(approved_target/ttd_target, 4)
        app_pct_control= round(approved_control/ttd_control, 4)
        air[prefix+var+p_target_list]= [round(app_pct_target/app_pct_control, 4)]
        
        print({'variable': var, 'ttd_target': ttd_target, 'ttd_control': ttd_control, 
               'approved_target': approved_target, 'approved_control': approved_control})
    print(air)
        
    return air

def air_calc_new(pred,prefix):
    air = {}
    target_control_dict = [
            {'age_cuts': {
                'target': ['Age 62 and over'],
                'control': ['Age below 62']
            }},
            {'Race': {
                'target': ['black', 'hispanic'],
                'control': ['white']
            }},
            {'Race': {
                'target': ['black'],
                'control': ['white']
            }},
            {'Race': {
                'target': ['hispanic'],
                'control': ['white']
            }},
            {'Gender': {
                'target': ['F'],
                'control': ['M']
            }},
            # 'mmct_tag': {
            #     'target': ['mmct'],
            #     'control': ['non-mmct']
            # }
    ]

    for var in target_control_dict:
        for i in var:
            target_list = var[i]['target']
            control_list = var[i]['control']
            p_target_list= ','.join(target_list) if (
                (len(target_list)<= 1) and (('black' in target_list) or ('hispanic' in target_list))
            ) else ''
            
            ttd_target= pred[pred[i].isin(target_list)]['con_wt'].sum()
            approved_target= pred[pred[i].isin(target_list)]['approved'].multiply(pred[pred[i].isin(target_list)]['con_wt']).sum()
            ttd_control= pred[pred[i].isin(control_list)]['con_wt'].sum()
            approved_control= pred[pred[i].isin(control_list)]['approved'].multiply(pred[pred[i].isin(control_list)]['con_wt']).sum()
            app_pct_target= round(approved_target/ttd_target, 4)
            app_pct_control= round(approved_control/ttd_control, 4)
            air[prefix+i+p_target_list]= [round(app_pct_target/app_pct_control, 4)]
        
            print({'variable': i, 'target_list': p_target_list, 'ttd_target': ttd_target, 'ttd_control': ttd_control, 
                   'approved_target': approved_target, 'approved_control': approved_control})
            
    print(air)
        
    return air

In [53]:
# TTD['ApproveCount'].value_counts(dropna= False)

In [54]:
def get_prediction_2(model, data, data_h2o, target, cols_tuple, score_col= 'p1', weight= 'RECWGT', asc= False):
    if data_h2o!= None:
        predict = model.predict(data_h2o)
    else:
        predict = model.predict(h2o.H2OFrame(data))
    p =predict.as_data_frame()
    dd = data.loc[:, col_tuple_pass]
    dd = pd.DataFrame(dd)
    dd = dd.reset_index()
    dd['RECWGT'] = dd["wt2"]
    dd['conwt'] = 1
    dd[target] = dd[bad_col] 
    pred = pd.concat([dd.loc[:, cols_tuple + ['RECWGT', 'conwt']],p],axis=1)
    
    return pred

In [55]:
def objective(space):
    from sklearn.metrics import roc_auc_score
    classifier = H2OXGBoostEstimator(ntrees= int(space['n_estimators']),
                            max_depth = 2 ,
                            learn_rate = space['learning_rate'],
                            min_rows = space['min_child_weight'],
                            sample_rate = space['subsample'],
                            colsample_bylevel = space['colsample_bylevel'],
                            reg_alpha = space['reg_alpha'],
                            reg_lambda = space['reg_lambda'],
                            min_split_improvement = space['gamma'],
                            colsample_bytree = space['colsample_bytree'],
                            model_id="RI_Auto_LN_V2_Hopt_1",
                            distribution = "bernoulli",
                            stopping_metric = 'logloss',
                            seed=42)
    h2o.no_progress()
    classifier.train(y=bad_col,x =col_list,  training_frame=train_h2o, weights_column =wt_col )
    h2o.show_progress()
    pred = get_prediction_2(model= classifier, data= Train, data_h2o= train_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')
    ### Finding Cutoff
    y_true = pred[bad_col].values
    y_prob = pred['p1'].values
    weights = pred['RECWGT'].values
    auc_train = roc_auc_score(y_true, y_prob, sample_weight=weights)
    train_ks,train_gini = rnkOrderingN(pred, bad_col, 'conwt',"p1", 5)[1]['KS'], rnkOrderingN(pred, bad_col, 'conwt',"p1", 5)[1]['Gini']
    # train_ks_cw,train_gini_cw = rnkOrdering2_ks_code(pred, 'bad', "con_wt","p1", 5)

    pred = get_prediction_2(model= classifier, data= Test, data_h2o= test_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')
    ### Finding Cutoff
    y_true = pred[bad_col].values
    y_prob = pred['p1'].values
    weights = pred['RECWGT'].values
    auc_test = roc_auc_score(y_true, y_prob, sample_weight=weights)
    test_ks,test_gini = rnkOrderingN(pred, bad_col, 'conwt',"p1", 5)[1]['KS'], rnkOrderingN(pred, bad_col, 'conwt',"p1", 5)[1]['Gini']
    # test_ks_cw,test_gini_cw = rnkOrdering2_ks_code(pred, "bad", "con_wt","p1", 5)
    
    pred = get_prediction_2(model= classifier, data= Oot, data_h2o= oot_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')
    ### Finding Cutoff
    y_true = pred[bad_col].values
    y_prob = pred['p1'].values
    weights = pred['RECWGT'].values
    auc_oot = roc_auc_score(y_true, y_prob, sample_weight=weights)
    oot_ks,oot_gini = rnkOrderingN(pred, bad_col, 'conwt',"p1", 5)[1]['KS'], rnkOrderingN(pred, bad_col, 'conwt',"p1", 5)[1]['Gini']
    # test_ks_cw,test_gini_cw = rnkOrdering2_ks_code(pred, "bad", "con_wt","p1", 5)

    # print('*'*15, ' TTD ', '*'*15)
    # pred= get_predictions(ttd_h2o, bad_col, wt_col, classifier)
    # pred = Equal_Approval_Cutoff_Barsath(pred)
    # display(pd.DataFrame(pred[['p1']].describe()).T)
    # air_ttd = air_calc_new(pred,'AIR ttd ')

    KS_diff = abs(train_ks-test_ks)
    gini_diff = abs(train_gini-test_gini)
    # ks_diff_nw = abs(train_ks_cw-test_ks_cw)
    # gini_diff_nw = abs(train_gini_cw-test_gini_cw)
    
    # ks_diff_normal= (ks_diff_nw)/15
    # ks_normal= (test_ks_cw - 20)/(40-20)
    auc_diff = abs(auc_train-auc_test)#/((auc_train+auc_test)/2)
    Stability = auc_diff
    Accuracy = (1-auc_test)
    # Fairness_Race = 1-air_ttd['AIR ttd Race'][0]
#     loss= (auc_diff*0.3) + ((1-auc_test)*0.2) + ((1-air_ttd['AIR ttd Race'][0])*0.5)
    loss= KS_diff
    
    metrics = {'loss': [loss], 'KS_diff': [KS_diff], #'KS diff non-wt': [ks_diff_nw],
                   'GINI dif': [gini_diff], #'GINI diff non-wt': [gini_diff_nw],
                   'KS train wt': [train_ks], #'KS train non-wt': [train_ks_cw],
                   'KS test wt': [test_ks], #'KS test non-wt': [test_ks_cw],
                   'KS oot wt': [oot_ks], #'KS oot non-wt': [oot_ks_cw],
#                    'AIR train': [air_tr], 'AIR test': [air_te], #'AIR oot': [air_oo],
                     # 'AIR ttd': [air_ttd],
               'AUC train':[auc_train], 'AUC test':[auc_test], #'AUC oot':[auc_oot]
    #                'Approval train target': [app_pct_target_tr], 'Approval train control': [app_pct_control_tr],
    #                'Approval test target': [app_pct_target_te], 'Approval test control': [app_pct_control_te],
    #                'Approval oot target': [app_pct_target_oo], 'Approval oot control': [app_pct_control_oo]
                  }
#     metrics.update(air_tr)
#     metrics.update(air_te)
    # metrics.update(air_oo)
    # metrics.update(air_ttd)

    # clear previous output
#     clear_output(wait=True)
#     metrics_df = pd.concat([metrics_df, pd.DataFrame(metrics)])
    # display(pd.DataFrame(metrics))
#     print(f'{loss}= ({auc_diff}*0.3) + ((1-{auc_test})*0.2) + ((1-{air_ttd["AIR ttd Race"][0]})*0.5))')
                
    return_dict =  {'loss': loss, 'status': STATUS_OK ,
           'KS_diff':KS_diff,'train_ks':train_ks,'test_ks':test_ks,'oot_ks':oot_ks, 
           'gini_diff':gini_diff,'train_gini':train_gini,'test_gini':test_gini,'oot_gini':oot_gini,
           # 'KS_cw_diff':ks_diff_nw,'train_ks_cw':train_ks_cw,'test_ks_cw':test_ks_cw,#'oot_ks_cw':oot_ks_cw,
           # 'gini_diff_nw':gini_diff_nw,'train_gini_cw':train_gini_cw,'test_gini_cw':test_gini_cw,#'oot_gini_cw':oot_gini_cw,
#            'AIR_train': air_tr, 'AIR_test': air_te, 'AIR_oot': air_oo, 'AIR ttd': air_ttd,
            'AUC train':auc_train,'AUC test':auc_test,'AUC oot':auc_oot,
             'Stability':Stability, 'Accuracy':Accuracy, 
                    # 'Fairness_Race':Fairness_Race
            }
#     return_dict.update({k: v[0] for k, v in air_tr.items()})
#     return_dict.update({k: v[0] for k, v in air_te.items()})
    # return_dict.update({k: v[0] for k, v in air_oo.items()})
    # return_dict.update({k: v[0] for k, v in air_ttd.items()})
    print(metrics)
    print()
    return return_dict

In [56]:
# space = {
#     'max_depth' : 2,#hp.choice('max_depth', range(1, 3, 1)),
#     'learning_rate' : hp.quniform('learning_rate',0.01,0.2,0.01),
#     'n_estimators' : ho_scope.int(hp.quniform('n_estimators',500,3000,100)),    
#     'min_child_weight' : hp.quniform('min_child_weight', 20, 150, 20),
#     'subsample' : hp.quniform('subsample', 0.4,1,0.1),
#     'colsample_bytree' : 1,#hp.quniform('colsample_bytree', 0.4, 1, 0.1),
#     'reg_lambda': hp.quniform('reg_lambda',1,20,0.5),
#     'reg_alpha': hp.quniform('reg_alpha',0,20,0.5),
#         'gamma':hp.quniform('gamma',0,20,0.25),
#             'colsample_bylevel':hp.quniform('colsample_bylevel',0.4, 1, 0.1)}

In [57]:
space = {
    'max_depth' :2, #  hp.choice('max_depth', range(2, 3, 1)),
    'learning_rate' : hp.quniform('learning_rate',0.01,0.2,0.001),
    'n_estimators' : ho_scope.int(hp.quniform('n_estimators',800,2500,100)),
    'min_child_weight' : hp.quniform('min_child_weight', 25, 300, 5),
    'subsample' : hp.quniform('subsample', 0.1, 0.6, 0.05),
    'colsample_bytree' : hp.quniform('colsample_bytree', 0.5, 1, 0.05),
    'reg_lambda':  hp.quniform('reg_lambda',5,30,0.5),
    'reg_alpha': hp.quniform('reg_alpha',1,15,0.5),
    'gamma':hp.quniform('gamma',5,35,0.5),
    'colsample_bylevel':hp.quniform('colsample_bylevel',0.5, 1, 0.05),
#     'seed':42
}

In [59]:
trials = Trials()

In [ ]:
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=75,
            trials=trials)

print("Best: ", best)

xgboost prediction progress: |                        
███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |                        
███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |                        
███████████████████████████████████████████████████| (done) 100%
{'loss': [0.28999999999999915], 'KS_diff': [0.28999999999999915], 'GINI dif': [0.7899999999999991], 'KS train wt': [33.5], 'KS test wt': [33.21], 'KS oot wt': [33.01], 'AUC train': [0.7338196356230444], 'AUC test': [0.7292819124740586]}
xgboost prediction progress: |                                                      
███████████████████████████████████████████████████| (done) 100%                    
xgboost prediction progress: |                                                      
███████████████████████████████████████████████████| (done) 100%                    
xgboost prediction progress: |                                 

In [ ]:
tr_results = pd.DataFrame(trials.results).reset_index()
tr_results['iter'] = tr_results['index']+1
# tr_results.head()

tr_vals = pd.DataFrame(trials.vals).reset_index()
tr_vals['iter'] = tr_vals['index']+1
# tr_vals.head()

trials_full = pd.merge(tr_vals.drop('index',axis=1),tr_results.drop('index',axis=1),how='left',on='iter')
trials_full.shape

(75, 25)

In [ ]:
trials_full

,colsample_bylevel,colsample_bytree,gamma,learning_rate,min_child_weight,n_estimators,reg_alpha,reg_lambda,subsample,iter,loss,status,KS_diff,train_ks,test_ks,oot_ks,gini_diff,train_gini,test_gini,oot_gini,AUC train,AUC test,AUC oot,Stability,Accuracy
0,0.80,0.75,8.5,0.128,260.0,1900.0,11.5,8.5,0.45,1,0.29,ok,0.29,33.50,33.21,33.01,0.79,44.24,43.45,43.35,0.733820,0.729282,0.729255,0.004538,0.270718
1,0.55,0.60,6.0,0.170,120.0,1800.0,13.5,15.5,0.15,2,0.19,ok,0.19,32.95,32.76,32.42,0.54,43.48,42.94,42.56,0.730102,0.726025,0.725635,0.004077,0.273975
2,0.95,0.90,16.5,0.099,245.0,2400.0,14.5,21.5,0.15,3,0.23,ok,0.23,30.70,30.93,30.72,0.03,40.18,40.21,40.16,0.711335,0.710566,0.711314,0.000769,0.289434
3,0.70,0.90,28.5,0.140,80.0,1500.0,3.5,21.0,0.10,4,0.31,ok,0.31,30.53,30.84,31.06,0.18,40.07,40.25,40.44,0.711955,0.712775,0.714073,0.000819,0.287225
4,0.55,0.80,16.5,0.162,220.0,1500.0,4.5,18.5,0.25,5,0.02,ok,0.02,32.11,32.13,32.03,0.09,42.09,42.00,41.91,0.722475,0.721580,0.722108,0.000895,0.278420
5,0.85,0.85,16.5,0.064,245.0,1900.0,12.0,25.0,0.40,6,0.01,ok,0.01,32.18,32.19,32.18,0.32,42.32,42.00,42.10,0.723574,0.722142,0.722957,0.001432,0.277858
6,0.60,0.65,5.0,0.110,255.0,1600.0,14.0,21.0,0.40,7,0.42,ok,0.42,33.65,33.23,33.30,0.81,44.34,43.53,43.46,0.734743,0.729489,0.729788,0.005253,0.270511
7,0.90,0.70,18.0,0.114,195.0,2300.0,2.5,12.5,0.45,8,0.01,ok,0.01,32.60,32.61,32.43,0.22,42.86,42.64,42.75,0.726470,0.724364,0.725455,0.002107,0.275636
8,0.80,0.60,8.5,0.169,215.0,2200.0,4.0,26.0,0.50,9,0.72,ok,0.72,34.08,33.36,33.36,1.29,44.98,43.69,43.79,0.737767,0.730516,0.730967,0.007250,0.269484
9,0.55,0.95,31.5,0.131,150.0,1100.0,10.0,10.0,0.55,10,0.20,ok,0.20,31.69,31.89,31.73,0.15,41.65,41.50,41.72,0.720035,0.719531,0.720554,0.000503,0.280469


In [66]:
# trials_full.to_csv(f'/root/Unified_LN_V2/LN_Unsec_V2/Model_Data/RI_Unsec_Model/090424_LN_{product}_RI_Hopt_1.csv', index=False)

In [67]:
# trials_full.query('train_ks_cw > 33 and train_ks_cw > test_ks_cw and train_ks_cw - test_ks_cw < 1.5').sort_values('AIR ttd Race',ascending=False)[[
#     'iter', 'KS_cw_diff', 'train_ks_cw', 'test_ks_cw', #'oot_ks_cw',
#       'gini_diff_nw', 'train_gini_cw', 'test_gini_cw',# 'oot_gini_cw',
#          'AIR ttd age_cuts', 'AIR ttd Race', 'AIR ttd Gender', 'AIR ttd Raceblack', 'AIR ttd Racehispanic',
#       #  'AIR ttd mmct_tag',
#        'Stability', 'Accuracy', 'Fairness_Race',
#       #  'Fairness_Age', 'Fairness_Gender'#, 'Fairness_MMCT_tag'
#        ]].query('`AIR ttd Race` >= 0.9 and Stability < 0.017 and Accuracy <= 0.225').sort_values('Accuracy')

In [69]:
# trials_full[(trials_full['KS_diff']<= 3.5) & (trials_full['KS_cw_diff']<= 2.5) #& 
# #             (np.abs(trials_full['train_ks']-trials_full['test_ks'])<= 1.5) &
# #             (np.abs(trials_full['train_ks_cw']-trials_full['test_ks_cw'])<= 1.0)
#            ].sort_values(by= ['train_ks_cw'], ascending= False)#.iloc[0]

In [70]:
final_trials = trials_full.sort_values(by=['train_ks','loss'], ascending=[False,True])

In [71]:
iter_chosen = 50
best_from_hpt = final_trials.query(f'iter == {iter_chosen}').iloc[0,:].to_dict()
# for item in best_from_hpt.keys():
#     print(best_from_hpt[item])
#     try:
#         best_from_hpt[item]= np.round(best_from_hpt[item], 2)
#     except:
#         continue
print(best_from_hpt)

{'colsample_bylevel': 0.8, 'colsample_bytree': 0.9500000000000001, 'gamma': 27.5, 'learning_rate': 0.11, 'min_child_weight': 190.0, 'n_estimators': 1900.0, 'reg_alpha': 1.0, 'reg_lambda': 14.5, 'subsample': 0.45, 'iter': 50, 'loss': 0.0, 'status': 'ok', 'KS_diff': 0.0, 'train_ks': 32.01, 'test_ks': 32.01, 'oot_ks': 32.07, 'gini_diff': 0.23000000000000398, 'train_gini': 42.03, 'test_gini': 41.8, 'oot_gini': 42.059999999999995, 'AUC train': 0.7219358854015485, 'AUC test': 0.7210667762308542, 'AUC oot': 0.7220406008294761, 'Stability': 0.0008691091706942711, 'Accuracy': 0.27893322376914575}


# best hpt

In [72]:
# train_h2o[bad_col] = train_h2o[bad_col].asfactor()
# # col_list = var_imp_list_baseline1 #[i for i in cols if i not in ["bad_flag","unique_id","wt2","client"]]
# # target = bad_col

In [73]:
len(col_list)

73

In [74]:
# best_from_hpt

In [75]:
xgbm_ri_best_hopt = H2OXGBoostEstimator(ntrees= int(best_from_hpt['n_estimators']),
                            max_depth = 2 ,
                            learn_rate = best_from_hpt['learning_rate'],
                            min_rows = best_from_hpt['min_child_weight'],
                            sample_rate = best_from_hpt['subsample'],
                            colsample_bylevel = best_from_hpt['colsample_bylevel'],
                            reg_alpha = best_from_hpt['reg_alpha'],
                            reg_lambda = best_from_hpt['reg_lambda'],
                            min_split_improvement = best_from_hpt['gamma'],
                            colsample_bytree = best_from_hpt['colsample_bytree'],
                            model_id=f"xgb_{product}_RI_best_hopt",
                            distribution = "bernoulli",
                            stopping_metric = 'logloss',
                            seed=42)

In [76]:
xgbm_ri_best_hopt.train(y=bad_col,x =col_list,  training_frame=train_h2o, weights_column =wt_col )

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: xgb_Unsec_RI_best_hopt


Model Summary: 
    number_of_trees
--  -----------------
    1900

ModelMetricsBinomial: xgboost
** Reported on train data. **

MSE: 0.20961183068774022
RMSE: 0.457833846157905
LogLoss: 0.6032786840145784
Mean Per-Class Error: 0.38197277992800904
AUC: 0.7219327714596367
AUCPR: 0.7204694051472644
Gini: 0.4438655429192735

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.3725284263862306
       0      1       Error    Rate
-----  -----  ------  -------  ------------------
0      23087  52866   0.696    (52866.0/75953.0)
1      5885   80774   0.0679   (5885.0/86659.0)
Total  28972  133640  0.3613   (58751.0/162612.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.372528     0.733312  276
max f2                       0.201271     0.862704  348
max f0point5                 0.52895      0.682856  188
max accuracy                 0.504491     0.666212  203
max precision                0.897024     1         0
max recall                   0.0638548    1         380
max specificity              0.897024     1         0
max absolute_mcc             0.469943     0.328235  223
max min_per_class_accuracy   0.557516     0.658394  171
max mean_per_class_accuracy  0.52895      0.661415  188
max tns                      0.897024     75953     0
max fns                      0.897024     86658     0
max fps                      0.0159451    75953     399
max tps                      0.0638548    86659     380
max tnr                      0.897024     1         0
max fnr                      0.897024     0.999988  0
max fpr                      0.0159451    1         399
max tpr                      0.0638548    1         380

Gains/Lift Table: Avg response rate: 53.29 %, avg score: 53.25 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100054                   0.829245           1.62388   1.62388            0.865396         0.843626  0.865396                    0.843626            0.0162476       0.0162476                  62.388    62.388             0.0133642
2        0.0200047                   0.812097           1.59141   1.60765            0.848093         0.820211  0.856748                    0.831922            0.0159129       0.0321605                  59.1412   60.7651            0.0260252
3        0.0300039                   0.7994             1.54179   1.5857             0.821648         0.805536  0.84505                     0.823129            0.0154167       0.0475773                  54.1789   58.5701            0.0376238
4        0.0400032                   0.788536           1.52909   1.57155            0.814883         0.793764  0.83751                     0.815789            0.0152898       0.0628671                  52.9094   57.1552            0.0489506
5        0.0500025                   0.778714           1.4887    1.55498            0.793358         0.783558  0.82868                     0.809343            0.0148859       0.077753                   48.8703   55.4984            0.0594127
6        0.100005                    0.740307           1.44767   1.50133            0.771492         0.758342  0.800086                    0.783843            0.0723872       0.15014                    44.7672   50.1328            0.107337
7        0.150001                    0.712305           1.40538   1.46935        

In [77]:
get_risk_rank(model= xgbm_ri_best_hopt, data= Train, data_h2o= train_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
162612 86659.0 1


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS
0,0.688206,0.897024,0.748395,32523,7520.0,25003.0,76.88,20.0,9.90,28.85,18.95
1,0.606670,0.688204,0.647527,32522,11073.0,21449.0,65.95,40.0,24.48,53.60,29.12
2,0.511625,0.606670,0.560871,32523,14024.0,18499.0,56.88,60.0,42.94,74.95,32.01
3,0.389331,0.511619,0.454437,32522,18028.0,14494.0,44.57,80.0,66.68,91.68,25.00
4,0.010490,0.389331,0.251165,32522,25308.0,7214.0,22.18,100.0,100.00,100.00,0.00


{'Gini': 42.03, 'KS': 32.01}


In [78]:
get_risk_rank(model= xgbm_ri_best_hopt, data= Test, data_h2o= test_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
69691 37126.0 1


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS
0,0.688525,0.886774,0.748988,13939,3272.0,10667.0,76.53,20.0,10.05,28.73,18.68
1,0.606997,0.688504,0.647878,13938,4711.0,9227.0,66.20,40.0,24.51,53.59,29.08
2,0.511426,0.606982,0.561114,13938,6003.0,7935.0,56.93,60.0,42.95,74.96,32.01
3,0.389786,0.511426,0.455064,13938,7776.0,6162.0,44.21,80.0,66.83,91.56,24.73
4,0.011797,0.389783,0.252428,13938,10803.0,3135.0,22.49,100.0,100.00,100.00,0.00


{'Gini': 41.8, 'KS': 32.01}


In [79]:
get_risk_rank(model= xgbm_ri_best_hopt, data= Oot, data_h2o= oot_h2o, target= 'approve_flag', cols_tuple= col_tuple_pass, score_col= 'p1', weight= 'conwt')

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
58076 31097.0 1


,min_score,max_score,mean_score,total,Goods,Bads,Bad_Rate,Cum_Total,Cum_Good,Cum_Bad,KS
0,0.687932,0.890351,0.747807,11616,2660.0,8956.0,77.10,20.0,9.86,28.80,18.94
1,0.605311,0.687929,0.646686,11615,3940.0,7675.0,66.08,40.0,24.46,53.48,29.02
2,0.510737,0.605305,0.559160,11615,4955.0,6660.0,57.34,60.0,42.83,74.90,32.07
3,0.389845,0.510737,0.454368,11615,6400.0,5215.0,44.90,80.0,66.55,91.67,25.12
4,0.009781,0.389841,0.252274,11615,9024.0,2591.0,22.31,100.0,100.00,100.00,0.00


{'Gini': 42.059999999999995, 'KS': 32.07}


In [80]:
var_imp_bst = xgbm_ri_best_hopt._model_json['output']['variable_importances'].as_data_frame()
# print(var_imp)
var_imp_bst['Var_unique'] = var_imp_bst['variable'].str.split('.', expand=True)[0]
var_imp_list_bst = list(var_imp_bst['Var_unique'].unique())

print(var_imp_list_bst, len(var_imp_list_bst))

['addrinputcountyratio', 'inquirynonshortterm12month', 'sourcevoterregistration', 'inquiryshortterm12month', 'addrinputphonecount', 'subjectrecordtimeoldest', 'derogtimenewest', 'assetpropcurrenttaxtotal', 'assetownership', 'addrinputsubjectcount', 'addrinputtaxvalue', 'assetpropevercount', 'addrinputavmvalue60month', 'sourcecredheadertimeoldest', 'inquiryauto12month', 'addronfilecount', 'addrinputavmvalue', 'inquirybanking12month', 'addrcurrentavmvalue60month', 'addrcurrentcountyratio', 'addrcurrentavmvalue', 'addrinputlengthofres', 'addrcurrenttaxvalue', 'derogcount12month', 'addrinputtimeoldest', 'addrinputdeedmailing', 'alertregulatorycondition', 'addrcurrentavmvalue12month', 'addrinputavmvalue12month', 'bankruptcytimenewest', 'addrcurrentavmratio60monthprior', 'sourcenonderogcount', 'assetproppurchasetimenewest', 'lienjudgmentdollartotal', 'addrlastmoveecontrajectory', 'lienjudgmenttaxcount', 'addrcurrentdeedmailing', 'addrprevioustimenewest', 'addrprevioustimeoldest', 'addrinputa

In [81]:
var_imp_bst

,variable,relative_importance,scaled_importance,percentage,Var_unique
0,addrinputcountyratio,15997.415039,1.000000,0.245058,addrinputcountyratio
1,inquirynonshortterm12month,8014.918457,0.501013,0.122777,inquirynonshortterm12month
2,sourcevoterregistration,6189.868164,0.386929,0.094820,sourcevoterregistration
3,inquiryshortterm12month,4093.400391,0.255879,0.062705,inquiryshortterm12month
4,addrinputphonecount,3872.540039,0.242073,0.059322,addrinputphonecount
5,subjectrecordtimeoldest,3053.252686,0.190859,0.046772,subjectrecordtimeoldest
6,derogtimenewest,2007.635254,0.125497,0.030754,derogtimenewest
7,assetpropcurrenttaxtotal,1900.843750,0.118822,0.029118,assetpropcurrenttaxtotal
8,assetownership,1649.392456,0.103104,0.025266,assetownership
9,addrinputsubjectcount,1632.477905,0.102046,0.025007,addrinputsubjectcount


In [82]:
var_imp_df = var_imp_bst.groupby('Var_unique').agg({'percentage':'sum'}).sort_values(by='percentage',ascending=False).reset_index()

In [83]:
var_imp_df

,Var_unique,percentage
0,addrinputcountyratio,0.245058
1,inquirynonshortterm12month,0.122777
2,sourcevoterregistration,0.094820
3,inquiryshortterm12month,0.062705
4,addrinputphonecount,0.059322
5,subjectrecordtimeoldest,0.046772
6,derogtimenewest,0.030754
7,assetpropcurrenttaxtotal,0.029118
8,assetownership,0.025266
9,addrinputsubjectcount,0.025007


### Saving All Prediction Data

In [85]:
Train['p1']= xgbm_ri_best_hopt.predict(train_h2o).as_data_frame()['p1']
Test['p1']= xgbm_ri_best_hopt.predict(test_h2o).as_data_frame()['p1']
Oot['p1']= xgbm_ri_best_hopt.predict(oot_h2o).as_data_frame()['p1']

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [86]:
Train['p1'].describe(), Test['p1'].describe(), Oot['p1'].describe()

(count    162612.000000
 mean          0.532481
 std           0.181176
 min           0.010490
 25%           0.424998
 50%           0.562674
 75%           0.667164
 max           0.897024
 Name: p1, dtype: float64,
 count    69691.000000
 mean         0.533097
 std          0.181083
 min          0.011797
 25%          0.425745
 50%          0.563208
 75%          0.667725
 max          0.886774
 Name: p1, dtype: float64,
 count    58076.000000
 mean         0.532063
 std          0.180607
 min          0.009781
 25%          0.425078
 50%          0.560686
 75%          0.666572
 max          0.890351
 Name: p1, dtype: float64)

In [87]:
Train['approve_flag'].mean(), Test['approve_flag'].mean(), Oot['approve_flag'].mean()

(0.5329188497773842, 0.5327230201891205, 0.5354535436324815)

In [88]:
col_tuple_pass

['unique_id',
 'client',
 'wt2',
 'fico_pb',
 'approve_flag',
 'funded_flag',
 'bad_flag',
 'Race',
 'Gender',
 'age_cuts']

In [89]:
Train['weight']= 1/Train['p1']
Test['weight']= 1/Test['p1']
Oot['weight']= 1/Oot['p1']

Train['weight'].mean(), Test['weight'].mean(), Oot['weight'].mean()

(2.9928935450957037, 2.994861164670315, 2.999035755461176)

In [92]:
pd.concat([
    pd.DataFrame(Train['weight'].describe()).T,
    pd.DataFrame(Test['weight'].describe()).T,
    pd.DataFrame(Oot['weight'].describe()).T
]
)

,count,mean,std,min,25%,50%,75%,max
weight,162612.0,2.992894,5.392507,1.114797,1.498882,1.777227,2.352953,95.330931
weight,69691.0,2.994861,5.403440,1.127683,1.497622,1.775543,2.348826,84.767470
weight,58076.0,2.999036,5.446848,1.123152,1.500213,1.783529,2.352509,102.239967


In [93]:
df_scored= pd.concat(
    [Train[['unique_id', 'client', 'fico_pb', 'approve_flag', 'funded_flag', 'bad_flag', 'Race', 'Gender', 'age_cuts', 'p1', 'weight']],
    Test[['unique_id', 'client', 'fico_pb', 'approve_flag', 'funded_flag', 'bad_flag', 'Race', 'Gender', 'age_cuts', 'p1', 'weight']],
    Oot[['unique_id', 'client', 'fico_pb', 'approve_flag', 'funded_flag', 'bad_flag', 'Race', 'Gender', 'age_cuts', 'p1', 'weight']]]
).reset_index(drop= True)

df_scored.shape

(290379, 11)

In [95]:
df_scored.head()

,unique_id,client,fico_pb,approve_flag,funded_flag,bad_flag,Race,Gender,age_cuts,p1,weight
0,107884,CPM,523.0,1.0,1.0,0.0,missing,F,Age below 62,0.434682,2.300530
1,SYM662540,Numerica,615.0,1.0,1.0,0.0,white,F,Age below 62,0.525862,1.901641
2,528377,Bethpage,695.0,0.0,NaN,NaN,black,F,Age below 62,0.588850,1.698224
3,457977,Bethpage,802.0,0.0,NaN,NaN,white,F,Age below 62,0.573168,1.744690
4,SYM522559,Numerica,690.0,1.0,1.0,0.0,white,M,Age below 62,0.644484,1.551628


In [96]:
df_scored.to_csv('/root/Unified_LN_V2/LN_Unsec_V2/Model_Data/RI_Unsec_Model/090424_RI_Auto_Model_Scored_Data.csv', index= False)

In [97]:
# xgbm_ri_best_hopt.download_mojo('/root/Unified_LN_V2/LN_Unsec_V2/Model_Data/RI_Unsec_Model/')

'/root/Unified_LN_V2/LN_Unsec_V2/Model_Data/RI_Unsec_Model/xgb_Unsec_RI_best_hopt.zip'

Closing connection _sid_9b3c at exit
H2O session _sid_9b3c closed.


In [98]:
# model= h2o.import_mojo('/root/Unified_LN_V2/LN_Auto_V2/Model_Data/RI_Auto_Model/xgb_auto_RI_best_hopt.zip')

In [99]:
# model.predict(train_h2o.head(4))

In [100]:
# del model